In [1]:
# Importing Libraries
import time
import cv2
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:

model = load_model("mymodel.h5", compile=False)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.FalseNegatives(), 'acc'])

In [3]:
with open('./category2label.pkl', 'rb') as cat:
    catrgory_2_label = pickle.load(cat)


In [4]:
print(catrgory_2_label)

img_size = (100, 100)

box_colors = {0: (0, 0, 255), 1: (0, 255, 0), 2: (0, 255, 255)}

{0: 'without_mask', 1: 'with_mask', 2: 'mask_weared_incorrect'}


In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [6]:
# video from webcam

cap = cv2.VideoCapture(0)
begin_time = time.time()
frame_cnt = 0

while cap.isOpened():
    rate, frame = cap.read()

    if not rate:
        break

    frame_cnt += 1
    frame = cv2.flip(frame, 1)
    face = face_cascade.detectMultiScale(frame, 1.3, 5)

    for (x, y, w, h) in face:
        # prediction

        roi = frame[y:y+h, x:x+w]
        data = cv2.resize(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB), img_size)
        data = data / 255.
        data = data.reshape((1,) + data.shape)

        pred = model.predict(data)

        target = np.argmax(pred, axis=1)[0]

        # drawing box
        cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=box_colors[target], thickness=2)
        # cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h),color=box_colors[target], thickness=2.5)
        text = "{}: {:.2f}".format(catrgory_2_label[target], pred[0][target])
        cv2.putText(frame, text, org=(x, y-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.6, color=(255, 255, 255), thickness=1)

    endtime = time.time()-begin_time
    fps = frame_cnt / endtime

    cv2.putText(img=frame, text="FPS: "+str(round(fps, 2)), org=(10, 20),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,  fontScale=0.5, color=(255, 255, 255), thickness=1)
    # Show the frame
    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 37ms/step
